## FASE 1. Extracción de datos de películas

In [5]:
# importamos todas las librerias necesarias para realizar el ejercicio

import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np

In [15]:
# extraemos los datos de la API y la guardamos en una variable (en formato JSON)

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url)
datos.status_code
datos.content

datos_pelis = datos.json()

## FASE 2. Creación de la base de datos

In [ ]:
# Iniciamos la connexion con SQL ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',   
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [9]:
# Creamos la base de datos 

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS films_ejercicio_2"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


In [ ]:
# para crear las tablas primero debemos revisar los datos del API y analizar que tablas y columnas debemos crear.
# Para este ejercicio, he decidido crear 4 tablas: 1. tabla de peliculas 2. tabla de generos 3. tabla de subtitulos 4. tabla intermedia peliculas_subtitulos

In [10]:
mycursor.execute("USE films_ejercicio_2")

query = '''CREATE TABLE genero (
    id_genero INT PRIMARY KEY AUTO_INCREMENT,
    genero VARCHAR(255) NOT NULL  
);'''

mycursor.execute(query)

In [11]:
mycursor.execute("USE films_ejercicio_2")
mycursor.execute("DROP TABLE IF EXISTS subtitulos")

query = '''CREATE TABLE subtitulos (
    id_idioma INT PRIMARY KEY AUTO_INCREMENT,
    idioma VARCHAR(255) NOT NULL  
);'''

mycursor.execute(query)

In [12]:

mycursor.execute("USE films_ejercicio_2")

mycursor.execute("DROP TABLE IF EXISTS films") #añado el drop table if exists por si tengo que volver a ejecutar el codigo con alguna modificacion

query = '''CREATE TABLE films (
    id_film INT PRIMARY KEY AUTO_INCREMENT,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion FLOAT,
    id_genero INT, 
    adultos TINYINT(1),
    FOREIGN KEY (id_genero) REFERENCES genero(id_genero)


);'''

mycursor.execute(query)

In [13]:
mycursor.execute("USE films_ejercicio_2")
mycursor.execute("DROP TABLE IF EXISTS peliculas_subtitulos")
query = '''CREATE TABLE peliculas_subtitulos (
    id_film INT,
    id_idioma INT,
    PRIMARY KEY (id_film, id_idioma),
    FOREIGN KEY (id_film) REFERENCES films(id_film),
    FOREIGN KEY (id_idioma) REFERENCES subtitulos(id_idioma)   
);'''

mycursor.execute(query)

## Fase 3. Inserción de los Datos en la Base de datos

## Insertamos datos en tabla subtitulos

In [16]:
# Repetimos el proceso para los idiomas en los subtitulos, en este caso, 
# dado que los elementos estan en lista, debemos hacer 2 bucles for

datos_subtitulos = []
for i in datos_pelis:
    for subtitulos in i['subtitulos']:
        datos_subtitulos.append(subtitulos)

#repetimos el proceso para dejar solo los valores unicos
lista_subtitulos = list(set(datos_subtitulos))
lista_subtitulos

['pt', 'ko', 'de', 'en', 'jp', 'es', 'fr', 'it']

In [17]:
for i in lista_subtitulos:
    query = "INSERT INTO subtitulos (idioma) VALUES (%s)"
    mycursor.execute(query, (i,))

## Insertamos datos en tabla genero

In [18]:
#Encontramos los valores que existen en genero
datos_genero = []
for i in datos_pelis:
    datos_genero.append(i['genero'])

# Nos quedamos con los valores únicos. Creando una set y luego lo pasamos a lista de nuevo.
lista_genero = list(set(datos_genero))
lista_genero


['Thriller',
 'Aventura',
 'Western',
 'Terror',
 'Fantasía',
 'Bélico',
 'Animación',
 'Drama',
 'Suspense',
 'Ciencia ficción',
 'Romance',
 'Musical',
 'Comedia',
 'Crimen',
 'Biografía',
 'Acción']

In [19]:
for i in lista_genero:
    query = "INSERT INTO genero (genero) VALUES (%s)"
    mycursor.execute(query, (i,))

## Insertamos datos en tabla films

In [20]:
datos_limpios_films = []

for peli in datos_pelis:
    # Obtener id_genero
    sql = "SELECT id_genero FROM genero WHERE genero = %s"
    mycursor.execute(sql, (peli['genero'],))
    id_genero = mycursor.fetchone()[0]

    # Crear la fila lista para insertar
    fila = (
        peli['titulo'],
        peli['año'],
        peli['duracion'],
        id_genero,               
        int(peli['adultos'])
    )

    datos_limpios_films.append(fila)


    mycursor.executemany("""
    INSERT INTO films (titulo, año, duracion, id_genero, adultos)
    VALUES (%s, %s, %s, %s, %s)
""", datos_limpios_films)
    
    cnx.commit()

## insertamos datos en tabla intermedia pelicula_subtitulos

In [21]:
mycursor.execute("USE films_ejercicio_2")

for peli in datos_pelis:

    # Buscar id_film por título (porque el título es único en tu dataset)
    mycursor.execute("SELECT id_film FROM films WHERE titulo = %s", (peli['titulo'],))
    id_film = mycursor.fetchone()[0]

    # Insertar subtítulos en la tabla puente
    for sub in peli['subtitulos']:
        mycursor.execute(
            "SELECT id_idioma FROM subtitulos WHERE idioma = %s", 
            (sub,)
        )
        id_idioma = mycursor.fetchone()[0]

        mycursor.execute("""
            INSERT INTO peliculas_subtitulos (id_film, id_idioma)
            VALUES (%s, %s)
        """, (id_film, id_idioma))

cnx.commit()

InternalError: Unread result found

In [ ]:
#Una vez terminamos el flujo de trabajo de inserción. Si dejamos de trabajar usando la connexion con SQL. Podriamos cerrarla.
mycursor.close()
cnx.close()
print('Conexión cerrada')

Conexión cerrada


NOTA_Posibles mejoras:
- Crear tabla de idiomas para los subtitulos?
- Crear tabla de genero (son datos que se repiten)

## FASE 4. Obtener información a partir de los datos.



In [ ]:
# Si hemos cerrado la connexion con SQL annteriormente. Ahora debemos iniciarla de nuevo ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
# Dado que hemos creado el Database en el apartado anterior. podemos ya abrir 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',    
        database = 'films_ejercicio_2'
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [47]:
mycursor = cnx.cursor()
mycursor.execute("USE films_ejercicio_2") # no haría falta si en el punto anterior hemos realizado la connexión ya usando este database

¿Cuántas películas tienen una duración superior a 120 minutos?

In [48]:
query = """
SELECT COUNT(*)
FROM films
WHERE duracion > 120
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(0,)]

¿Cuántas películas incluyen subtítulos en español?

In [55]:
query = """
SELECT COUNT(*)
FROM films
WHERE subtitulos LIKE "%ES%";
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(100,)]

¿Cuántas películas tienen contenido adulto?

In [56]:
query = """
SELECT COUNT(*)
FROM films
WHERE adultos = 1;
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(47,)]

¿Cuál es la película más antigua registrada en la base de datos?

In [66]:
query = """
SELECT titulo, año
FROM films
ORDER BY año
LIMIT 1
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[('Citizen Kane', 1941)]

Muestra el promedio de duración de las películas agrupado por género.

In [67]:
query = """
SELECT ROUND(AVG(duracion),2) AS promedio_duracion, genero
FROM films
GROUP BY genero;
"""

#En este caso muestro el resultado en un data frame asi vemos las columnas y sus cabeceras
df = pd.read_sql(query, cnx)
df


C:\Users\laura\AppData\Local\Temp\ipykernel_31080\1018756946.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,promedio_duracion,genero
0,154.29,Crimen
1,126.26,Drama
2,139.44,Acción
3,136.31,Ciencia ficción
4,139.67,Romance
5,161.00,Bélico
6,121.67,Thriller
7,128.00,Musical
8,159.80,Fantasía
9,133.00,Aventura


¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

In [68]:
query = """
SELECT count(id) AS total_peliculas, año
FROM films
GROUP BY año
ORDER BY total_peliculas DESC
"""


df = pd.read_sql(query, cnx)
df


C:\Users\laura\AppData\Local\Temp\ipykernel_31080\2953748035.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,total_peliculas,año
0,5,2001
1,4,2013
2,4,1994
3,4,2008
4,4,1999
5,4,2017
6,3,2010
7,3,1998
8,3,2014
9,3,2000


In [71]:
query = """
SELECT año
FROM films
GROUP BY año
ORDER BY COUNT(*) DESC
LIMIT 1;
"""


mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(2001,)]

Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

In [72]:
query = """
SELECT genero, count(id)
FROM films
GROUP BY genero
"""


df = pd.read_sql(query, cnx)
df

C:\Users\laura\AppData\Local\Temp\ipykernel_31080\469406158.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,genero,count(id)
0,Crimen,7
1,Drama,27
2,Acción,9
3,Ciencia ficción,13
4,Romance,3
5,Bélico,2
6,Thriller,6
7,Musical,1
8,Fantasía,5
9,Aventura,3


Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).

In [73]:
query = """
SELECT titulo
FROM films
WHERE titulo LIKE '%GODFATHER%';
"""


mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[('The Godfather',), ('The Godfather Part II',)]